## CÓDIGO IMDB DRAMA

In [1]:
import pandas as pd

from bs4 import BeautifulSoup
import requests
import json
import csv
import random
import re

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

from time import sleep
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
with open ('Fase1_datos/lista_comedia.json', 'r') as fichero:
    lista_comedia = json.load (fichero)

In [4]:
len(lista_comedia)

4588

In [184]:
busqueda = lista_comedia[4501:5001]

In [7]:
# SOLO EJECUTAR UNA VEZ
info_por_peli = []
fallos = []

In [185]:
# Acceder a la web mediante la Url y maximizar:
driver = webdriver.Chrome()
driver.get('https://www.imdb.com/')
driver.maximize_window()
sleep(random.randint(1,3))


# Rechazar cookies:
driver.find_element('css selector', '#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
sleep(random.randint(1,3))

contador = 0

# Entrando en cada título
for peli in busqueda:
    lista_info = [f'{peli[0]}',]
    contador +=1
    print(f'Película: {contador}')

    try:
        
        # Metemos el ID en el buscador
        try:
            driver.find_element('css selector', '#suggestion-search').send_keys(peli[0], Keys.ENTER)
            sleep(random.randint(1,3))

            # Extraemos info
            info = driver.find_element('xpath' , f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section').text
            lista_info.append(info.split('\n'))

            info_por_peli.append(lista_info)
        except:
            fallos.append(peli)
            driver.find_element('css selector', '#error > div.error_attrib > a').click()
        
                    
    except:
        continue

Película: 1
Película: 2
Película: 3
Película: 4
Película: 5
Película: 6
Película: 7
Película: 8
Película: 9
Película: 10
Película: 11
Película: 12
Película: 13
Película: 14
Película: 15
Película: 16
Película: 17
Película: 18
Película: 19
Película: 20
Película: 21
Película: 22
Película: 23
Película: 24
Película: 25
Película: 26
Película: 27
Película: 28
Película: 29
Película: 30
Película: 31
Película: 32
Película: 33
Película: 34
Película: 35
Película: 36
Película: 37
Película: 38
Película: 39
Película: 40
Película: 41
Película: 42
Película: 43
Película: 44
Película: 45
Película: 46
Película: 47
Película: 48
Película: 49
Película: 50
Película: 51
Película: 52
Película: 53
Película: 54
Película: 55
Película: 56
Película: 57
Película: 58
Película: 59
Película: 60
Película: 61
Película: 62
Película: 63
Película: 64
Película: 65
Película: 66
Película: 67
Película: 68
Película: 69
Película: 70
Película: 71
Película: 72
Película: 73
Película: 74
Película: 75
Película: 76
Película: 77
Película

In [9]:
# SOLO LA PRIMERA VEZ
lista_tuplas = []

In [182]:
# Recogida de los datos
for peli in info_por_peli:
    
    patron = re.compile(r'([a-z])([A-Z])')
    patron_duracion = re.compile(r'\b(?:\d+h\s*)?\d+min\b')
    tupla_datos = [peli[0],] 

    # Listas intermedias para ir checkeando
    direccion = []
    guion = []
    nota = []
    duracion = []
    argumento = []
        
    for indice, linea in enumerate(peli[1]):
               
        
        # Directores    
        if 'Dirección' in linea:
                                
            directores = patron.sub(r'\1, \2', peli[1][indice+1])
            direccion.append(directores)            


        # Guionistas
        elif 'Guión' in linea:
            
            guionistas = patron.sub(r'\1, \2', peli[1][indice+1])
            guion.append(guionistas)
                
        
        # Nota y duracion
        elif 'PUNTUACIÓN EN IMDb' in linea:
            nota.append(float(peli[1][indice+1].replace(',', '.')))
            
            if r'\d+h' or r'\d+min' in peli[1][indice-1]:  
                duracion.append(peli[1][indice-1])
            
                    
        # Argumento
        elif 'Añade un argumento en tu idioma' in linea:
            argumento.append(peli[1][indice+1])

    # Añadimos a la tupla los datos de la lista intermedia
    for elemento in [direccion, guion, nota, duracion, argumento]:
        if elemento != []:
            tupla_datos.append(elemento[0])
        else:
            tupla_datos.append(np.nan)
    
    lista_tuplas.append(tupla_datos)

In [183]:
# Creacion de Dataframe
df = pd.DataFrame(lista_tuplas)

# Guarda la informacion recopilada en un documento csv
df.to_csv('InfoComediaBien.csv', index=False)
